In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.linear_model import Ridge,ElasticNet,Lasso,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [2]:
df = pd.read_csv('/Users/suryasaikadali/Downloads/pw_skills/end_to_End_projects/student_performance/artifacts/data.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [5]:
x = df.drop('math_score',axis = 1)
y = df['math_score']

In [6]:
cat_cols = [i for i in x.columns if df[i].dtype=='object']
num_cols = [i for i in x.columns if df[i].dtype=='int64']

In [7]:
cat_cols

['gender',
 'race_ethnicity',
 'parental_level_of_education',
 'lunch',
 'test_preparation_course']

In [8]:
num_cols

['reading_score', 'writing_score']

In [9]:
num_pipe = StandardScaler()
cat_pipe = OneHotEncoder(sparse_output=False,drop='first')

In [10]:
preprocessor = ColumnTransformer([
    ('num',num_pipe,num_cols),
    ('cat',cat_pipe,cat_cols)
  
])
 

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [12]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns = preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns = preprocessor.get_feature_names_out())

In [13]:
x_train

,num__reading_score,num__writing_score,cat__gender_male,cat__race_ethnicity_group B,cat__race_ethnicity_group C,cat__race_ethnicity_group D,cat__race_ethnicity_group E,cat__parental_level_of_education_bachelor's degree,cat__parental_level_of_education_high school,cat__parental_level_of_education_master's degree,cat__parental_level_of_education_some college,cat__parental_level_of_education_some high school,cat__lunch_standard,cat__test_preparation_course_none
0,-1.594536,-0.988448,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.857519,0.628865,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.333479,-0.516732,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.543656,0.022373,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,-0.473597,-0.786284,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2.118575,2.111403,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
746,-0.473597,-0.516732,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
747,-0.543656,-1.055836,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
748,-1.524478,-1.055836,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
models = {
    'linear regressor':LinearRegression(),
    'random forest Regressor':RandomForestRegressor(),
    'xgboost Regressor':XGBRegressor(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'elastic net':ElasticNet()
}

In [15]:
for i,j in models.items():
    print(i)
    print('&&&&&&')
    print(j)

linear regressor
&&&&&&
LinearRegression()
random forest Regressor
&&&&&&
RandomForestRegressor()
xgboost Regressor
&&&&&&
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)
ridge
&&&&&&
Ridge()
lasso
&&&&&&
Lasso()
elastic net
&&&&&&
ElasticNet()


In [16]:
def evaluate_model(true,predicted):
    mse = mean_squared_error(true,predicted)
    mae = mean_absolute_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [17]:
r2_list = []

In [18]:
for key,values in models.items():
    model = values.fit(x_train,y_train)
        
    y_pred = model.predict(x_test)

    mae,rmse,r2 = evaluate_model(y_test,y_pred)
    r2_list.append(f'{key}:{r2}')    
    print(f'The rsquare score of {key} model is {r2}')
    print(f'The mae  of {key} model is {mae}')
    print(f'The rmse of {key} model is {rmse}')
    print('%%%%%%%%%%%%%%%')
    print('\n')
        

The rsquare score of linear regressor model is 0.8778243107659014
The mae  of linear regressor model is 4.337930600167403
The rmse of linear regressor model is 5.4825281236597725
%%%%%%%%%%%%%%%


The rsquare score of random forest Regressor model is 0.8486070792953212
The mae  of random forest Regressor model is 4.825926666666668
The rmse of random forest Regressor model is 6.102970252262417
%%%%%%%%%%%%%%%


The rsquare score of xgboost Regressor model is 0.8337628841400146
The mae  of xgboost Regressor model is 5.038006618499756
The rmse of xgboost Regressor model is 6.39517545811892
%%%%%%%%%%%%%%%


The rsquare score of ridge model is 0.877873971198214
The mae  of ridge model is 4.3365196569328015
The rmse of ridge model is 5.481413775990447
%%%%%%%%%%%%%%%


The rsquare score of lasso model is 0.820248418531758
The mae  of lasso model is 5.218405738075969
The rmse of lasso model is 6.650048754307145
%%%%%%%%%%%%%%%


The rsquare score of elastic net model is 0.7174880104782451
Th